# Imports

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
logs = Path('/mnt/logs')
batch_logs = logs.joinpath('batch_delaunay')
batch_logs_v0 = batch_logs.joinpath('v0_foxp3')
print(batch_logs_v0.exists())
grep_errors = [fn for fn in batch_logs.glob('*.txt')]
print(grep_errors)
# err = pd.read_csv(grep_errors[1],sep=':',header=None)
pd.read_csv(grep_errors[1],sep='#1',header=None)
err.head()

True
[PosixPath('/mnt/logs/batch_delaunay/TypeError_v0.txt'), PosixPath('/mnt/logs/batch_delaunay/foxp3_v0.txt')]


<ipython-input-6-968617bd85ff>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  pd.read_csv(grep_errors[1],sep='#1',header=None)


,0,1
0,-rw-r--r-- 1 bri8 rbao 3.9M Jun 8 11,"11 133_042420 #01 TMA_Phenotyping #1_Core[1,8..."
1,-rw-r--r-- 1 bri8 rbao 1.1M Jun 8 11,"03 28_042420 #02 TMA_Phenotyping #2_Core[1,12..."
2,-rw-r--r-- 1 bri8 rbao 2.1M Jun 8 11,"07 31_042420 #02 TMA_Phenotyping #2_Core[1,12..."
3,-rw-r--r-- 1 bri8 rbao 880K Jun 8 11,"04 33_042420 #02 TMA_Phenotyping #2_Core[1,12..."


In [60]:
mnt_point = '/ix/rbao/Projects/HCC-CBS-028-Pfizer-TBruno'
for log in err.iloc[:,0]:
    log_path = batch_logs_v0.joinpath(log.split('/')[1])
    print('\n',log_path)
    with open(str(log_path)) as file:
        content = file.readlines()
        jobid = content[0].split(': ')[1][:-1]
        temp = content[3].split(mnt_point)[1]
        raw_fn = temp.split(' to ')[0]
        tma = raw_fn.split('/')[4]
        fn_only = raw_fn.split('/')[-1]
        print(jobid,tma,fn_only,sep=',')
        


 /mnt/logs/batch_delaunay/v0/slurm-926808_13.out
13,TMA01,042420  #01 TMA_Phenotyping #1_Core[1,1,R]_[62880,6954]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926910_102.out
102,TMA02,042420  #02 TMA_Phenotyping #2_Core[1,6,R]_[65626,9454]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926910_34.out
34,TMA02,042420  #02 TMA_Phenotyping #2_Core[1,2,C]_[41529,4452]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926808_71.out
71,TMA01,042420  #01 TMA_Phenotyping #1_Core[1,3,S]_[64644,10320]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926808_8.out
8,TMA01,042420  #01 TMA_Phenotyping #1_Core[1,1,L]_[53821,6914]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926808_149.out
149,TMA01,042420  #01 TMA_Phenotyping #1_Core[1,9,I]_[49132,19419]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926910_84.out
84,TMA02,042420  #02 TMA_Phenotyping #2_Core[1,5,O]_[60624,7723]_cell_seg_data.txt

 /mnt/logs/batch_delaunay/v0/slurm-926808_123.out
123,TMA01,0424

# Compare input files to results and see what is missing

In [67]:
inputs = pd.Series(['_'.join(fn.parts[-1].split('_')[0:3]) \
              for fn in data_pn.glob('*.txt')])
inputs = list(inputs.values)
inputs.sort()
print('\n'.join(inputs[0:3]))

042420  #02 TMA_Phenotyping #2_Core[1,10,C]
042420  #02 TMA_Phenotyping #2_Core[1,10,D]
042420  #02 TMA_Phenotyping #2_Core[1,10,E]


In [7]:
tmas = ['TMA01','TMA02']
results = Path('/mnt/results/neighborhoods/v0/')
result_fns = pd.Series(['_'.join(fn.parts[-1].split('_')[1:4]) \
              for fn in results.glob('*.csv')])
missing=[]
jobids=[]
for tma in tmas:
    data_pn = Path('/mnt/data/TBruno_TMA_Phenotyping/Cell_segmentation/'+tma)
    inputs = pd.Series(['_'.join(fn.parts[-1].split('_')[0:3]) \
                  for fn in data_pn.glob('*.txt')])
    inputs = list(inputs.values)
    inputs.sort()
    for jobid,input_fn in enumerate(inputs):
        if not np.any(result_fns.isin([input_fn]).values):
            missing.append(input_fn)
            jobids.append(jobid)
for miss,jobid in zip (missing,jobids):
    print(miss,jobid)

In [9]:
result_fns[0:2]

0    042420  #01 TMA_Phenotyping #1_Core[1,4,O]
1    042420  #01 TMA_Phenotyping #1_Core[1,3,I]
dtype: object